In [2]:
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM
from keras.layers import Conv1D, Flatten, MaxPooling1D

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [4]:
vocab_size = 1000
max_len =1000
batch_size = 32
embedding_dims =10
filters = 16
ker_size = 3 # kernel size
hidden_dims = 250
epochs = 10

In [71]:
data = pd.read_csv('spacy.csv', encoding= 'latin')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747 entries, 0 to 746
Data columns (total 7 columns):
sentence_id    201 non-null float64
Word           747 non-null object
Tag            747 non-null object
Category       747 non-null int64
Unnamed: 4     0 non-null float64
Unnamed: 5     0 non-null float64
Unnamed: 6     1 non-null object
dtypes: float64(3), int64(1), object(3)
memory usage: 40.9+ KB


In [72]:
data['Is M'] = 0
data['Is S'] = 0
data['Is Do'] = 0
data['Is Du'] = 0
data['Is O'] = 0
for i in range(0, 746):
    if(data['Category'][i]==1):
        data['Is M'][i]=1
    if(data['Category'][i]==2):
        data['Is S'][i]=1
    if(data['Category'][i]==3):
        data['Is Do'][i]=1
    if(data['Category'][i]==4):
        data['Is Du'][i]=1
    if(data['Category'][i]==5):
        data['Is O'][i]=1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A v

In [73]:
x = data.sample(frac=1)
x

,sentence_id,Word,Tag,Category,Unnamed: 4,Unnamed: 5,Unnamed: 6,Is M,Is S,Is Do,Is Du,Is O
142,51.0,tab,MED,1,NaN,NaN,NaN,1,0,0,0,0
725,NaN,I,O,5,NaN,NaN,NaN,0,0,0,0,1
401,NaN,ml,STR,2,NaN,NaN,NaN,0,1,0,0,0
229,NaN,once,DOS,3,NaN,NaN,NaN,0,0,1,0,0
376,NaN,mg,STR,2,NaN,NaN,NaN,0,1,0,0,0
220,NaN,once,DOS,3,NaN,NaN,NaN,0,0,1,0,0
471,NaN,Combiflam,MED,1,NaN,NaN,NaN,1,0,0,0,0
533,NaN,Crocin,MED,1,NaN,NaN,NaN,1,0,0,0,0
518,NaN,Azithromycin,MED,1,NaN,NaN,NaN,1,0,0,0,0
361,NaN,ml,STR,2,NaN,NaN,NaN,0,1,0,0,0


In [74]:
x.to_csv('dosage.csv')

In [5]:
data = pd.read_csv('dosage.csv')

In [6]:
y = pd.DataFrame(data[['Is M','Is S','Is Do', 'Is Du', 'Is O']])
x_train,x_test,y_train,y_test = train_test_split(data['Word'], y,random_state=0)


In [7]:
tokenizer = text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(x_train)

In [8]:
x_train = tokenizer.texts_to_matrix(x_train)
x_test = tokenizer.texts_to_matrix(x_test)

In [9]:
x_train = sequence.pad_sequences(x_train,maxlen=max_len)
x_test = sequence.pad_sequences(x_test,maxlen=max_len)

In [10]:
model = Sequential()
# First we add an embedding layer 
model.add(Embedding(vocab_size,embedding_dims,input_length=max_len)) 
# Adding a 1D convolutional Layer
model.add(Conv1D(filters, ker_size, padding='valid', activation='relu'))
# Max Pooling the Convolutions
model.add(MaxPooling1D())
# Again Computing the Convolutions
model.add(Flatten())
model.add(Dense(hidden_dims, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
model.fit(x_train,y_train, batch_size=batch_size, epochs=15, validation_data=(x_test, y_test),)

Instructions for updating:
Use tf.cast instead.
Train on 560 samples, validate on 187 samples
Epoch 1/15
560/560 [==============================] - 3s 4ms/step - loss: 0.5291 - acc: 0.7846 - val_loss: 0.5044 - val_acc: 0.8000
Epoch 2/15
560/560 [==============================] - 1s 2ms/step - loss: 0.4951 - acc: 0.8004 - val_loss: 0.5042 - val_acc: 0.8000
Epoch 3/15
560/560 [==============================] - 1s 2ms/step - loss: 0.4854 - acc: 0.8004 - val_loss: 0.4937 - val_acc: 0.8000
Epoch 4/15
560/560 [==============================] - 1s 2ms/step - loss: 0.4761 - acc: 0.8004 - val_loss: 0.4727 - val_acc: 0.8000
Epoch 5/15
560/560 [==============================] - 1s 2ms/step - loss: 0.4489 - acc: 0.8004 - val_loss: 0.4442 - val_acc: 0.8000
Epoch 6/15
560/560 [==============================] - 1s 2ms/step - loss: 0.3973 - acc: 0.8132 - val_loss: 0.3879 - val_acc: 0.8332
Epoch 7/15
560/560 [==============================] - 1s 2ms/step - loss: 0.3243 - acc: 0.8518 - val_loss: 0.3202 

In [82]:
model.evaluate(x_test,y_test)[1]*100  # Accuracy of the model

187/187 [==============================] - 0s 380us/step


95.18716354421116

In [107]:
import pickle
pickle.dump(model,open('cnn_model.pkl','wb'))

In [108]:
pickle.dump(tokenizer,open('tokenizer.pkl','wb'))

In [103]:
s = 'I prescribe Combiflam 200 mg for 3 days, twice.'
s = s.split()
sen = s
s = pd.Series(s)
s= tokenizer.texts_to_matrix(s)
s = sequence.pad_sequences(s)
l = model.predict(s)

In [104]:
l

array([[6.84499741e-04, 6.91682100e-04, 1.81511343e-02, 1.20431185e-04,
        9.76879358e-01],
       [1.69128180e-04, 5.41299582e-04, 2.72528231e-02, 1.07000470e-02,
        9.66494858e-01],
       [9.17718351e-01, 5.68628311e-05, 2.03544497e-02, 2.30103731e-04,
        3.53401899e-02],
       [7.26436973e-02, 9.26887035e-01, 1.33615732e-03, 3.86497378e-03,
        2.07210183e-02],
       [8.15695524e-03, 9.98312593e-01, 5.42700291e-05, 1.78098679e-04,
        4.10944223e-04],
       [4.72203195e-02, 4.77731228e-05, 2.71379948e-04, 2.45600939e-04,
        9.68737721e-01],
       [4.73797321e-04, 1.64002180e-03, 1.30617619e-02, 9.71386552e-01,
        3.48189473e-02],
       [4.56930995e-02, 9.26405191e-04, 3.01462412e-03, 6.34885371e-01,
        2.93840587e-01],
       [8.49500939e-04, 1.27342599e-03, 9.89283979e-01, 1.76348202e-02,
        2.14207475e-03]], dtype=float32)

In [105]:
tags = ['Medicine','Strength', 'Dosage','Duration', 'Others']
d = dict()
for i in range(len(l)):
    x = np.argmax(l[i])
    print(x)
    if tags[x] not in d:
        d[tags[x]] =sen[i]
    else:
        s = d[tags[x]]
        s = (s,sen[i])
        d[tags[x]] = s

4
4
0
1
1
4
3
3
2


In [106]:
d

{'Others': (('I', 'prescribe'), 'for'),
 'Medicine': 'Combiflam',
 'Strength': ('200', 'mg'),
 'Duration': ('3', 'days,'),
 'Dosage': 'twice.'}

In [12]:
keras_file = 'modelname.h5'
model.save(keras_file)

In [ ]:
m